In [ ]:
from IPython.display import SVG, display

svg_code = """
<svg width="640" height="100" xmlns="http://www.w3.org/2000/svg">

  <defs> 
    <filter id="shadow"><feDropShadow dx="0.8" dy="0.8" stdDeviation="0.7" flood-color="#999999"/></filter> 
    <filter id="shadow2"><feDropShadow dx="1.5" dy="1.5" stdDeviation="1" flood-color="grey"/></filter> 
  </defs>
  
  <text x="280" y="45" 
        font-family="Georgia, serif" 
        font-size="32" 
        font-weight="bold" 
        fill="#1E428A"
        filter="url(#shadow)">
    <tspan>Mathematik II</tspan>
    <tspan x="280" dy="1.2em">für Ingenieure</tspan>
  </text>
  

  <g filter="url(#shadow2)" fill="#183176" clip-path="url(#clip0)" 
  transform="translate(-120 -340)"> 
  <path transform="scale(0.32)" 
  d="M2199.83 1278.62C2177.15 1278.62 
  2158.59 1260.11 2158.59 1237.5 2158.59 
  1214.89 2177.15 1196.38 2199.83 1196.38 
  2222.51 1196.38 2241.06 1214.89 2241.06 1237.5 
  2241.06 1260.11 2222.51 1278.62 2199.83 
  1278.62ZM2292.6 1211.8C2290.54 1204.26 2287.45 
  1197.07 2283.67 1190.56L2292.26 1164.86 2272.67 
  1145.33 2246.9 1153.89C2240.03 1150.13 2232.82 1147.04 
  2225.26 1144.99L2213.57 1121 2186.08 1121 2174.06 1144.99C2166.5 
  1147.04 2159.28 1150.13 2152.75 1153.89L2126.98 1145.33 2107.4 
  1164.86 2115.99 1190.56C2112.21 1197.41 2109.11 1204.61 2107.05 
  1212.14L2083 1223.79 2083 1251.21 2107.05 1263.2C2109.11 1270.74 
  2112.21 1277.93 2115.99 1284.44L2107.4 1310.14 2126.98 1329.67 2152.75 
  1321.11C2159.63 1324.88 2166.84 1327.96 2174.4 1330.01L2186.43 1354 
  2213.92 1354 2225.94 1330.01C2233.5 1327.96 2240.72 1324.88 2247.25 
  1321.11L2273.02 1329.67 2292.6 1310.14 2284.01 1284.44C2287.79 1277.59 
  2290.89 1270.39 2292.95 1262.86L2317 1250.86 2317 1223.45 2292.6 1211.8Z" 
  fill-rule="evenodd" />
  </g>
  
</svg>
"""

display(SVG(svg_code))

<br><br>

# 14 - Einführung in die numerische Lösung von    Differentialgleichungen mit digitalen Methoden

In [ ]:
def SlopeField(fig,ff,f,ls,dtg,dyg,trange,yrange):
    t = np.arange(trange[0],1.25*trange[1],dtg)
    y = np.arange(yrange[0],1.25*yrange[1],dyg)
    T, Y  = np.meshgrid(t, y)
   
    F  = f(T,Y)
    dt = ls/np.sqrt(1+F**2)
    dy = F*dt
    
    qfig = ff.create_quiver(T.flatten()-0/2*dt.flatten(), 
                            Y.flatten()-0/2*dy.flatten(),
                            dt.flatten(),dy.flatten(),
                            arrow_scale=0.01,
                            showlegend=False,hoverinfo='none',
                            line=dict(color='rgba(10,70,150,0.3)',width=1))
    
    for trace in qfig.data:
        fig.add_trace(trace)
                                 
    return fig

In [ ]:
# Import and Settings
#import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display, HTML
pio.templates.default = None
    
def plotODE(t,y,f):

    # Set up figure
    fig = go.Figure()
    
      
    # Create slider steps
    steps = []
    h_vals = [0.5,0.4,0.3, 0.2, 0.1]
    
    
    for i, h in enumerate(h_vals):
        is_visible = (i==2)
        
        # Get solutions
        t  = np.arange(0,2,h_vals[i])
        y_em = EulerMethod(t,f,y0,h)
    
        # Add traces
        fig.add_trace(go.Scatter(x=t, y=y_em, mode='lines', 
                                 name='y_i',showlegend=False,
                                 visible=is_visible,
                                 line=dict(color='#1E76B4')))
        fig.add_trace(go.Scatter(x=t, y=y_em, mode='markers', 
                                 name='y_i',showlegend=False,
                                 visible=is_visible,
                                 marker=dict(color='#3B7D23',size=10)))

      
   
    
    # Add slope field
    fig = SlopeField(fig,ff,f,0.5,0.1,0.1,[0,2],[0,2])
    
    # Create slider
    for i, h in enumerate(h_vals):
        visibility = [True] * (len(fig.data))
        visibility[0:2*len(h_vals)] = [False]*2*len(h_vals)
        # Set the i-th pair of traces to visible
        visibility[i*2] = True
        visibility[i*2 + 1] = True
        
        step = dict(
            method="restyle",
            args=[{"visible": visibility}],
            label=f"{h:.1f}"  # Label for the slider tick
        )
        steps.append(step)
     
    
    sliders = [dict(
        active=2,  # The slider starts at the first step (h=0.5)
        currentvalue={"prefix": "h = "},
        pad={"t": 50}, # Padding to not overlap with title
        steps=steps
    )]
        
    # Set layout
    fs = 16
    fig.update_layout(
        sliders=sliders,
        font=dict(size=fs),  # Font size
        legend=dict(xanchor="left", x=0.01),  # Legend alignment
        width=650,  # Figure width (example value)
        height=500,  # Figure height (example value)
        margin=dict(l=80, r=20, b=65, t=50),  # Margins
        xaxis=dict(
            range=[0, 2],  # Example x-axis range
            title=dict(text="t", font=dict(size=fs)),  # LaTeX-style title
            scaleanchor="x",  # Scale anchor
            tick0=0.2,  # Tick starting value
            dtick=0.2   # Tick interval
        ),
        yaxis=dict(
            range=[0, 2],  # Example y-axis range
            title=dict(text="y(t)", font=dict(size=fs)),  # LaTeX-style title
            tick0=0.2,  # Tick starting value
            dtick=0.2   # Tick interval
        )
    )

    display(HTML(f"""
    <div style="display: flex; justify-content: center; align-items: center;">
    {fig.to_html()}
    </div>
    """))

## 1. Das Euler-Verfahren

Verfahrensvorschrift: $\quad t_{i+1}=t_i+h, \quad y_{i+1}=y_i+h f(t_i,y_i)$

In [ ]:
import numpy as np

# ODE Function
def f(t,y):
    return t**2-y**2

# Euler's Method
def EulerMethod(t, f, y0, h):
    y    = np.zeros(len(t))
    y[0] = y0
    for i in range(len(t)-1):
        y[i+1] = y[i] + h * f(t[i], y[i])
    return y

# Parameters
h  = 0.3
t  = np.arange(0,2,h)
y0 = 1.0

# Solution
y_em = EulerMethod(t,f,y0,h)

In [ ]:
plotODE(t,y_em,f)

**Abbildung 1**: Simulationsergebnis des Euler Verfahrens über der Zeit.

<br><br>

In [ ]:
# ODE Function
def fStudy(t,y):
    return t**2-y**2

# Euler's Method
def EulerMethodStudy(t, f, y0, h):
    y    = np.zeros(len(t))
    y[0] = y0
    for i in range(len(t)-1):
        y[i+1] = y[i] + h * f(t[i], y[i])
    return y

# Simulation Study Euler
y0_study   = 1
h_study    = [0.5,0.4,0.3,0.2,0.1,0.01]
t_study    = []
y_em_study = []
for i,h in enumerate(h_study):
    t_study.append(np.arange(0,2,h))
    y_em_study.append(EulerMethodStudy(t_study[-1], fStudy, y0_study, h))

# Plot Function
def plotMultipleODEs(h_study,t_study,y_study,f,y0):

    # Set up figure
    fig = go.Figure()
    
    # Calculate the solutions
    for i,h in enumerate(h_study):
        # Add traces
        color=f'rgba(10,{140-10*i},{230-20*i},1.0)'
        fig.add_trace(go.Scatter(x=t_study[i], y=y_em_study[i], mode='lines', 
                                 name=f'h={h}',
                                 line=dict(color=color)))
        fig.add_trace(go.Scatter(x=t_study[i], y=y_em_study[i], mode='markers', 
                                 name='y_i',showlegend=False,
                                 marker=dict(color=color,size=10)))
    
    

    # Add slope field
    fig = SlopeField(fig,ff,f,0.2,0.05,0.05,[0.4,1.6],[0.4,1.6])
    
        
    # Set layout
    fs = 16
    fig.update_layout(
        font=dict(size=fs),  # Font size
        legend=dict(xanchor="left", x=0.01),  # Legend alignment
        width=650,  # Figure width (example value)
        height=500,  # Figure height (example value)
        margin=dict(l=80, r=20, b=65, t=50),  # Margins
        xaxis=dict(
            range=[0.6, 1.4],  # Example x-axis range
            title=dict(text="t", font=dict(size=fs)),  # LaTeX-style title
            scaleanchor="x",  # Scale anchor
            tick0=0.2,  # Tick starting value
            dtick=0.2   # Tick interval
        ),
        yaxis=dict(
            range=[0.4, 1.2],  # Example y-axis range
            title=dict(text="y(t)", font=dict(size=fs)),  # LaTeX-style title
            tick0=0.2,  # Tick starting value
            dtick=0.2   # Tick interval
        )
    )

    display(HTML(f"""
    <div style="display: flex; justify-content: center; align-items: center;">
    {fig.to_html()}
    </div>
    """))
    
def plotError(h_study,t_study,y_study):
        
    # Set up figure
    fig = go.Figure()
    
    # Calculate the errors
    e = []
    for i,h in enumerate(h_study):
        y_min:h=np.interp(t_study[i], t_study[-1], y_study[-1], 
                          left=None, right=None, period=None)
        e.append(np.max(np.absolute(y_min-y_study[i])))
    
    # Create trace
    fig.add_trace(go.Scatter(x=h_study, y=e, mode='lines', showlegend=False,
                                 line=dict(color='#C72426')))
    fig.add_trace(go.Scatter(x=h_study, y=e, mode='markers', showlegend=False,
                                 marker=dict(color='#C72426',size=10)))
    
    # Set layout
    fs = 16
    fig.update_layout(
        font=dict(size=fs),  # Font size
        legend=dict(xanchor="left", x=0.01),  # Legend alignment
        width=650,  # Figure width (example value)
        height=500,  # Figure height (example value)
        margin=dict(l=80, r=20, b=65, t=50),  # Margins
        xaxis=dict(
            range=[0, 0.5],  # Example x-axis range
            title=dict(text="h", font=dict(size=fs)),  # LaTeX-style title
            scaleanchor="x",  # Scale anchor
            autorange='reversed',
            tick0=0.1,  # Tick starting value
            dtick=0.1   # Tick interval
        ),
        yaxis=dict(
            range=[0, 0.25],  # Example y-axis range
            title=dict(text="e(h)", font=dict(size=fs)),  # LaTeX-style title
            tick0=0.05,  # Tick starting value
            dtick=0.05   # Tick interval
        )
    )
    
    # Show Plot
    display(HTML(f"""
    <div style="display: flex; justify-content: center; align-items: center;">
    {fig.to_html()}
    </div>
    """))
            
        

In [ ]:
plotMultipleODEs(h_study,t_study,y_em_study,fStudy,y0_study)

**Abbildung 2**: Verhalten der Näherung aus dem Euler Verfahren bei Reduktion der Schrittweite $h$.

In [ ]:
plotError(h_study,t_study,y_em_study)

**Abbildung 3:** Verhalten des Fehlers $e=||y_{0.01}-y_h||_\infty$  (Euler) beim Reduzieren der Schrittweite $h$.

Es lässt sich erkennen wie der **Fehler linear von** $\boldsymbol{h}$ **abhängt**. Aus diesem Grund bezeichnet man das **Euler Verfahren** als **Verfahren 1.Ordnung**.

<br><br>

## 2. Das Verfahren von Heun

Verfahrensvorschrift: $\quad t_{i+1}=t_i+h, \quad k_1=f(t_i,y_i), \quad k_2=f(t_i+h,y_i+h k_1),  \quad y_{i+1}=y_i + \frac{h}{2}(k_1+k_2)$

In [ ]:
def HeunMethod(t,f,y0,h):
    y    = np.zeros(len(t))
    y[0] = y0
    for i in range(len(t)-1):
        k1     = f(t[i], y[i])
        k2     = f(t[i]+h,y[i]+h*k1)
        y[i+1] = y[i] + h/2 * (k1+k2)
    return y

In [ ]:
def plotAndCompareODEs(y0,f):

    # Set up figure
    fig = go.Figure()
    
      
    # Create slider steps
    steps = []
    h_vals = [0.5,0.4,0.3, 0.2, 0.1]
    
    
    for i, h in enumerate(h_vals):
        is_visible = (i==2)
        
        # Get solutions
        t  = np.arange(0,2,h_vals[i])
        y_em = EulerMethodStudy(t,f,y0,h)
        y_he = HeunMethod(t,f,y0,h)
    
        # Add traces for Euler
 
        
        fig.add_trace(go.Scatter(x=t, y=y_em, mode='lines', 
                                 name='Euler',showlegend=True,
                                 visible=is_visible,
                                 line=dict(color='#1E76B4')))
        fig.add_trace(go.Scatter(x=t, y=y_em, mode='markers', 
                                 name='y_i',showlegend=False,
                                 visible=is_visible,
                                 marker=dict(color='#1E76B4',size=10)))
        
        # Add traces for Heun
        fig.add_trace(go.Scatter(x=t, y=y_he, mode='lines', 
                                 name='Heun',showlegend=True,
                                 visible=is_visible,
                                 line=dict(color='#C72426')))
        fig.add_trace(go.Scatter(x=t, y=y_he, mode='markers', 
                                 name='y_i',showlegend=False,
                                 visible=is_visible,
                                 marker=dict(color='#C72426',size=10)))

      
   
    
    # Add slope field
    fig = SlopeField(fig,ff,f,0.5,0.1,0.1,[0,2],[0,2])
    
    # Create slider
    for i, h in enumerate(h_vals):
        visibility = [True] * (len(fig.data))
        visibility[0:4*len(h_vals)] = [False]*4*len(h_vals)
        # Set the i-th pair of traces to visible
        visibility[i*4] = True
        visibility[i*4 + 1] = True
        visibility[i*4 + 2] = True
        visibility[i*4 + 3] = True
        
        step = dict(
            method="restyle",
            args=[{"visible": visibility}],
            label=f"{h:.1f}"  # Label for the slider tick
        )
        steps.append(step)
     
    
    sliders = [dict(
        active=2,  # The slider starts at the first step (h=0.5)
        currentvalue={"prefix": "h = "},
        pad={"t": 50}, # Padding to not overlap with title
        steps=steps
    )]
        
    # Set layout
    fs = 16
    fig.update_layout(
        sliders=sliders,
        font=dict(size=fs),  # Font size
        legend=dict(xanchor="left", x=0.01),  # Legend alignment
        width=650,  # Figure width (example value)
        height=500,  # Figure height (example value)
        margin=dict(l=80, r=20, b=65, t=50),  # Margins
        xaxis=dict(
            range=[0, 2],  # Example x-axis range
            title=dict(text="t", font=dict(size=fs)),  # LaTeX-style title
            scaleanchor="x",  # Scale anchor
            tick0=0.2,  # Tick starting value
            dtick=0.2   # Tick interval
        ),
        yaxis=dict(
            range=[0, 2],  # Example y-axis range
            title=dict(text="y(t)", font=dict(size=fs)),  # LaTeX-style title
            tick0=0.2,  # Tick starting value
            dtick=0.2   # Tick interval
        )
    )

    display(HTML(f"""
    <div style="display: flex; justify-content: center; align-items: center;">
    {fig.to_html()}
    </div>
    """))

In [ ]:
plotAndCompareODEs(y0_study,fStudy)

**Abbildung 4**: Simulationsergebnis des Verfahrens von Heun im Vergleich zum Euler Verfahren.

In [ ]:
# Simulation Study Heun
y_he_study = []
for i,h in enumerate(h_study):
    y_he_study.append(HeunMethod(t_study[i], fStudy, y0_study, h))

In [ ]:
plotError(h_study,t_study,y_he_study)

**Abbildung 5:** Verhalten des Fehlers $e=||y_{0.01}-y_h||_\infty$ (Heun) beim Reduzieren der Schrittweite $h$.

Man erkennt wie der **Fehler quadratisch von** $\boldsymbol{h}$ **abhängt**. Aus diesem Grund bezeichnet man das **Verfahren von Heun** als **Verfahren 2.Ordnung**.

<br><br>

## 3. Anwendungsbeispiel: Stick-Slip Schwingung

Differentialgleichung des Minimalmodells für reibungsindizierte Schwingungen nach Stribeck: 

$\begin{align} m\ddot{x}+cx= F_R \end{align}$ 

$\begin{align} F_R = -\left( F_C+(F_S-F_C)\exp\left(-\left|\frac{\dot{x}-v_B}{v_S}\right|^2\right)\right)\text{sign}(\dot{x}-v_B)+d(\dot{x}-v_B)\end{align}$

  $x(t)$: horizontale Auslenkung der Masse $m$, $c$: Federsteifigkeit, $F_C$: Gleitreibungskraft (Coulomb), $F_S$: Haftkraft (statisches Limit), $v_B$: Geschwindigkeit des Untergrunds.

Umgeschrieben in ein System erster Ordnung ($\dot{x}=v$, teilen durch $m$):

$\begin{align} \begin{bmatrix} \dot{x} \\ \dot{v} \end{bmatrix} = \begin{bmatrix} v \\ -\frac{c}{m} x + \frac{F_R}{m}\end{bmatrix} \end{align}$

In [ ]:
p = dict(cm=8,dm=0.25,vS=0.01,vB=0.05,Fc=1,Fs=1.1,)

def FR(v,p):
    return -((p["Fc"]+(p["Fs"]-p["Fc"])*
            np.exp(-np.absolute((v-p["vB"])/p["vS"])**2))*np.sign(v-p["vB"])
            +p["dm"]*(v-p["vB"]))

def fSL(t,y,p):
    dy = np.array([y[1], - p["cm"]*y[0] + FR(y[1],p)]) 
    return dy

def HeunMethodSystems(t,f,y0,h,p):
    n    = len(y0)
    y = np.zeros((len(t), n))
    y[0] = np.array(y0)
    for i in range(len(t)-1):
        k1     = f(t[i], y[i],p)
        k2     = f(t[i]+h,y[i]+h*k1,p)
        y[i+1] = y[i] + h/2 * (k1+k2)
    return y

In [ ]:
# Parameters
h_SL  = 0.001
t_SL  = np.arange(0,20.05,h_SL)
y0_SL = np.array([0.0,0.0])

# Solution
y_SL = HeunMethodSystems(t_SL,fSL,y0_SL,h_SL,p)

In [ ]:
def plotTrace(t,y,ylabel,yrange):
    fig = go.Figure()

    # Add traces
    fig.add_trace(go.Scatter(x=t, y=y, 
                         mode='lines',line=dict(color='#1E76B4')))

    # Set layout
    fs = 16
    fig.update_layout(
        font=dict(size=fs),  # Font size
        legend=dict(xanchor="left", x=0.01),  # Legend alignment
        width=650,  # Figure width (example value)
        height=250,  # Figure height (example value)
        margin=dict(l=80, r=20, b=65, t=50),  # Margins
        xaxis=dict(
            range=[t[0], t[-1]],  # Example x-axis range
            title=dict(text="t", font=dict(size=fs)),  # LaTeX-style title
            scaleanchor="x"
        ),
        yaxis=dict(
            range=yrange,  # Example y-axis range
            title=dict(text=ylabel, font=dict(size=fs))
        )
    )
    
    display(HTML(f"""
    <div style="display: flex; justify-content: center; align-items: center;">
    {fig.to_html()}
    </div>
    """))

In [ ]:
# Position Plot
plotTrace(t_SL,y_SL[:,0],"x(t)",[0,.2])

In [ ]:
# Velocity Plot
plotTrace(t_SL,y_SL[:,1],"v(t)",[-1.5*p["vB"],1.5*p["vB"]])

In [ ]:
plotTrace(t_SL,FR(y_SL[:,1],p),"FR(t)",[0,1.25])

**Abbildung 6:** Simulationsergebnis für die Stick-Slip Schwingung. <br>$\quad\quad\quad\quad\quad$**Oben:** Auslenkung $x(t)$, **mitte:** Geschwindigkeit $v(t)$, **unten:** Reibkraft $F_R(t)$.